# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686596


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:34,  3.26s/it]

Rendering models:  13%|█▎        | 4/31 [00:06<01:02,  2.31s/it]

Rendering models:  23%|██▎       | 7/31 [00:06<00:38,  1.62s/it]

Rendering models:  29%|██▉       | 9/31 [00:06<00:25,  1.15s/it]

...

Rendering models:  58%|█████▊    | 18/31 [00:07<00:04,  3.09it/s]

Rendering models:  68%|██████▊   | 21/31 [00:07<00:02,  4.14it/s]

Rendering models:  90%|█████████ | 28/31 [00:07<00:00,  5.71it/s]

Rendering models: 100%|██████████| 31/31 [00:07<00:00,  7.22it/s]

not-logged-in-20a275b53abec8db9f41    0.109953
Linda_J._Berkel                       0.000177
ElisabethB                            0.113393
not-logged-in-742cef5e1a82d2ddd0fb    0.000274
Camelron                              0.000433
shocko61                              0.000193
shocko61                              0.000148
kjkavene                              0.000224
mtajnai                               0.005577
jmfranci                              0.001772
justsomenoodles                       0.000203
Jboy2000                              0.002768
Jessica_Korkmaz                       0.002278
not-logged-in-5f30b2bb9b7f9d49da41    0.008474
shin_ai                               0.518860
Ellaida                               0.000309
pangeli5                              0.000701
Saharisunshine                        0.000246
aidenr                                0.000282
acapirala                             0.002489
mlee195                               0.002101
r.sundstrom3 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())